In [1]:
import os
from openai import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from operator import itemgetter

os.environ["OPENAI_API_KEY"] = ""
job_id = 'ftjob-hjLV1XIM6ccPjzx6zTpAYw5z'

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
finetuned_job = client.fine_tuning.jobs.retrieve(job_id)
finetuned_model = finetuned_job.fine_tuned_model

In [2]:
loader = PyPDFLoader("../_RAG_3.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=80)
docs = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(docs, embeddings)
db.save_local("_FAISS")

In [3]:
llm = ChatOpenAI(model_name=finetuned_model, temperature=0)
retriever = db.as_retriever()

In [4]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [5]:
template = """Your name is  Assistant. You are an assistant for golf related question-answering tasks. Use the context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Context : {context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [6]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [7]:
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | llm

In [15]:
conversational_qa_chain.invoke(
    {
        "question": "my name is berk",
        "chat_history": [
            HumanMessage(content="what are the games in ?"),
            AIMessage(content=" has games such as PuttPong, splash, maze, challangezone, shooter."),
        ],
    }
)


AIMessage(content='My name is  Assistant.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [1]:
!pip install ffmpeg

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6090 sha256=15165bb93f93a61b72d7df1d436ce78e96ba243c5868c6a2b408a9402ce51393
  Stored in directory: c:\users\berkg\appdata\local\pip\cache\wheels\8e\7a\69\cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: C:\Users\berkg\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip
